## shared

In [60]:
import numpy as np
def boxes_match(box1,box2,iou_threshold=0.4):
    
    if(box1[1]!=box2[1] or box1[7]==1 or box2[7]==1):
        return False
    
    return iou(box1[3:7],box2[3:7])>0.4

def boxes_equal(box1,box2,iou_threshold=0.6):
   
    if(box1[1]!=box2[1] or box1[2]!=box2[2]):
        return False
    
    return iou(box1[3:7],box2[3:7])==1
def box_matches_list(box1,list_boxes):
    index =-1
    if(len(list_boxes)==0):
        return index
    match_list = []
    box_index =0
    
    for box in list_boxes:
       
        if(box.shape[0]>0):
            
            if(box.shape[0]==1):
                
                if(boxes_match(box[0],box1)):
                    match_list.append(box_index)
                    
            elif(box.shape[0]==8):
                
                if(boxes_match(box,box1)):
                    
                    match_list.append(box_index)
        box_index+=1
    
    if(len(match_list)== 0):
        return index
    max_match_index = 0
    max_match_iou = 0
    for i in match_list:
        cur_iou = iou(box1[3:7],list_boxes[i][3:7])
        
        if(cur_iou>max_match_iou):
            max_match_iou=cur_iou
            max_match_index =i
    list_boxes[max_match_index][7] = 1
    box1[7] = 1
    
    return max_match_index

def box_in_list(box1,list_boxes):
    if(len(list_boxes)==0):
        return -1
    index=0
    for box in list_boxes:
        if(box.shape[0]>0):
            if(box.shape[0]==1):
                if(boxes_match(box[0],box1)):
                    return index
            elif(box.shape[0]==7):
                if(boxes_match(box,box1)):
                    return index
        index+=1
    return -1
def center(box):
    return np.array([(box[3]+box[5])/2,(box[4]+box[6])/2],np.float32)
def topleft(box):
    return np.array([box[3],box[4]],np.float32)
def botright(box):
    return np.array([box[5],box[6]],np.float32)
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

## Reading Text input of predictions

In [59]:
text_file_path = "ssd_300_baseline_modd.txt"
f = open(text_file_path,"r")
line = f.readline()
detections={}
comps = []
while(line):
    
    line = line.replace("\n", "")
    comps = line.split(",")
    print(comps)
    if(comps[0] in detections):
        detections[comps[0]].append(list(map(float, comps[2:])))
    else:
        detections[comps[0]]=[]
        detections[comps[0]].append(list(map(float, comps[2:])))
    line=f.readline()
f.close()
print(detections)
frame_width = 1032
frame_height = 778

['modd/00062.jpg', 'boat', '4', '0.689186', '-6.000000', '202.000000', '112.000000', '351.000000']
['modd/00063.jpg', 'boat', '4', '0.864713', '-6.000000', '197.000000', '126.000000', '344.000000']
['modd/00064.jpg', 'boat', '4', '0.916836', '-6.000000', '197.000000', '126.000000', '343.000000']
['modd/00065.jpg', 'boat', '4', '0.942657', '-8.000000', '195.000000', '130.000000', '343.000000']
['modd/00066.jpg', 'boat', '4', '0.914929', '-8.000000', '195.000000', '134.000000', '348.000000']
['modd/00067.jpg', 'boat', '4', '0.939819', '-6.000000', '196.000000', '141.000000', '351.000000']
['modd/00068.jpg', 'boat', '4', '0.905567', '-7.000000', '194.000000', '143.000000', '353.000000']
['modd/00069.jpg', 'boat', '4', '0.933746', '-13.000000', '191.000000', '143.000000', '346.000000']
['modd/00070.jpg', 'boat', '4', '0.925584', '-16.000000', '178.000000', '152.000000', '342.000000']
['modd/00071.jpg', 'boat', '4', '0.853593', '-17.000000', '172.000000', '156.000000', '340.000000']
['modd/

## No Tracking (Baseline)

In [57]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2 as cv
import time
import json
import numpy as np
from bounding_box_utils.bounding_box_utils import iou
frame_width = 640
frame_height = 464

no_tracking_res = [] 
firstFrame = None
frameCount =0
total_objects_tracking=0
out_tracking = cv.VideoWriter('retina.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
started = False
total_frames = 200
prev_frame=None

while frameCount<total_frames:
    
    frame = cv.imread('../modd/%s.jpg'%str(frameCount+1).zfill(5))
    
    frameCount+=1
    
    if frame is None:
        break
    if(frameCount<70):
        continue
        
    if 'modd/%s.jpg'%str(frameCount+1).zfill(5) in detections:
        for box in detections['modd/%s.jpg'%str(frameCount+1).zfill(5)]:
            if(box[0] == 8):
                xmin = int(box[2] )
                ymin = int(box[3])
                xmax =int(box[4])
                ymax =int(box[5])
                cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), (0, 255, 0), 2)
                cv.putText(frame,'{}: {:.2f}'.format(classes[int(box[0])], box[1]), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                no_tracking_res.append({"image_id" : frameCount, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : box[1]})
                total_objects_tracking+=1
        
  
    out_tracking.write(frame)
    
    frameCount+=1

out_tracking.release()
#with open('results_json/ssd_300_graal_baseline.json', 'w') as outfile:  
#    json.dump(no_tracking_res, outfile)
print(total_objects_tracking)


69


## BBox Center Flow

In [5]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2 as cv
import time
import json
from bounding_box_utils.bounding_box_utils import iou
feature_params = dict( maxCorners = 25,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
no_tracking_res = [] 
tracking_res = []

# initialize the first frame in the video stream
firstFrame = None
frameCount =0
step = 0.05
total_objects_no_tracking=0
total_objects_tracking=0
frame_width = 1032
frame_height = 778

preds = []
pred = None
tracks=[]
out_tracking = cv.VideoWriter('ssd_300_center_graal_1.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
started = False
# your code

multiplier=0
cc=0
prev_frame=None
total_frames=1100
frameCount=800
while frameCount<total_frames:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    frame = cv.imread('../graal_1/%s.jpg'%str(frameCount+1).zfill(5))
    
 
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    frameCount+=1
    if(frameCount<0):
        continue
   
    preds = []
    if 'graal_1/%s.jpg'%str(frameCount+1).zfill(5) in detections:
        
        for box in detections['graal_1/%s.jpg'%str(frameCount+1).zfill(5)]:
           
            if(box[0]!=4):

                continue
            temp_pred = np.insert(box,0,0)
            temp_pred=np.insert(temp_pred,7,0)
            preds.append(temp_pred)
   
    for track in tracks:
        track[7]=0
    
    
    for pred in preds:
        index = box_matches_list(pred,tracks)
        if(index>-1):
            
            multiplier = tracks[index][0]
            org_conf =tracks[index][2]
            
            if(multiplier<0):
                multiplier=1
            else:
                multiplier +=1
            
             
            del tracks[index]
            
            pred[2] += multiplier*step
            
            pred[0] = multiplier
            
            tracks.append(pred)
        else:
            
            temp_pred = np.copy(pred)
           
            multiplier = 1
            
          
            temp_pred[0] = multiplier
            temp_pred[7]=1
            tracks.append(temp_pred)
         
            cc = frameCount
    #print('current tracks')
    #print(tracks)
    for track in tracks:
        if(track[7]==0 and box_matches_list(track,preds)==-1):
            #print('not found in current frame')
            track[0]-=1
            #print('multiplier decreased to ',track[0])
            track[2]+=(step*track[0])
            frame_grey = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            prev_frame_grey = cv.cvtColor(prev_frame,cv.COLOR_BGR2GRAY)
            flow = cv.calcOpticalFlowFarneback(prev_frame_grey,frame_grey, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            xmin = int(track[3])
            ymin = int(track[4] )
            xmax =int(track[5] )
            ymax =int(track[6])
            center_flow = flow[np.minimum(int((xmin+xmax)/2),flow.shape[0]-1),np.minimum(int((ymin+ymax)/2),flow.shape[1]-1)]
          
            
            track[3] += center_flow[0]
            track[5]+=center_flow[1]
            track[4]+=center_flow[0]
            track[6]+=center_flow[1]
            
            
    to_display = [track for track in tracks if track[2]>0.4]
    #print(tracks)
    for box in to_display:
    # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
    
        xmin = int(box[3] )
        ymin = int(box[4] )
        xmax =int(box[5] )
        ymax =int(box[6] )
        cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), (0, 255, 0), 2)
        cv.putText(frame,'{}: {:.2f}'.format(classes[int(box[1])], box[2]), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        total_objects_tracking+=1
        tracking_res.append({"image_id" : frameCount, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : int(box[1])})
 
    out_tracking.write(frame)
    prev_frame=frame
# cleanup the camera and close any open windows

out_tracking.release()
print(total_objects_tracking)
with open('ssd_300_center_graal_1.json', 'w') as outfile:  
    json.dump(tracking_res, outfile)

378


In [21]:
out_tracking.release()

## BBox Keypoints flow

In [6]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2 as cv
import time
import json
from bounding_box_utils.bounding_box_utils import iou
feature_params = dict( maxCorners = 25,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
no_tracking_res = [] 
tracking_res = []

# initialize the first frame in the video stream
firstFrame = None
frameCount =0
step = 0.05
total_objects_no_tracking=0
total_objects_tracking=0

print(frame_width)
print(frame_height)
preds = []
pred = None
tracks=[]

out_tracking = cv.VideoWriter('ssd_300_keypoint_graal1.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
started = False
# your code
print("prev_count   current_count   entered   exited")
multiplier=0
cc=0
prev_frame=None

total_frames=1100
frameCount=800
while frameCount<total_frames:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    frame = cv.imread('../graal_1/%s.jpg'%str(frameCount+1).zfill(5))
    
 
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    
    if(frameCount<0):
        continue
   
    preds = []
    if 'graal_1/%s.jpg'%str(frameCount+1).zfill(5) in detections:
        
        for box in detections['graal_1/%s.jpg'%str(frameCount+1).zfill(5)]:
           
            if(box[0]!=4):

                continue
            temp_pred = np.insert(box,0,0)
            temp_pred=np.insert(temp_pred,7,0)
            preds.append(temp_pred)

    for track in tracks:
        track[7]=0
    #print(len(prev_pred),len(temp),len(entered_pred),len(exited_pred))
    
    for pred in preds:
        index = box_matches_list(pred,tracks)
      
        if(index>-1):
            
            multiplier = tracks[index][0]
            org_conf =tracks[index][2]
            if(multiplier<0):
                multiplier=1
            else:
                multiplier +=1
            
             
            tracks[index][2] = org_conf+ multiplier*step
            tracks[index][3] = pred[3]
            tracks[index][4] = pred[4]
            tracks[index][5] = pred[5]
            tracks[index][6] = pred[6]
            tracks[index][0] = multiplier
        else:
            
            temp_pred = np.copy(pred)
           
            multiplier = 1
           
          
            temp_pred[0] = multiplier
            temp_pred[7]=1
            tracks.append(temp_pred)
         
            cc = frameCount
  
    for track in tracks:
        if(track[7]==0):
            #print('not found in current frame')
            track[0]=-1
            #print('multiplier decreased to ',track[0])
            track[2]+=(step*track[0])
            frame_grey = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            prev_frame_grey = cv.cvtColor(prev_frame,cv.COLOR_BGR2GRAY)
            mask = np.zeros(frame_grey.shape, dtype = "uint8")


            cv.rectangle(mask, (int(track[3]), int(track[4])), (int(track[5]), int(track[6])), (255, 255, 255), -1)
            p0 = cv.goodFeaturesToTrack(prev_frame_grey, mask = mask, **feature_params)
            if(not p0 is None ):
                p1, st, err = cv.calcOpticalFlowPyrLK(prev_frame_grey, frame_grey, p0, None, **lk_params)
                
                average_flow = np.average(p1-p0,0)[0]
            else:
                average_flow=[0,0]
            flow_x_box_coords = int(average_flow[0])
            flow_y_box_coords = int(average_flow[1])
            track[3] += flow_x_box_coords
            track[5]+=flow_x_box_coords
            track[4]+=flow_y_box_coords
            track[6]+=flow_y_box_coords
            
            
    to_display = [track for track in tracks if track[2]>0.4]
    tracks= to_display
   
    for box in to_display:
    # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
    
        xmin = int(box[3])
        ymin = int(box[4])
        xmax =int(box[5])
        ymax =int(box[6])
        cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), (0, 255, 0), 2)
        cv.putText(frame,'{}: {:.2f}'.format(classes[int(box[1])], box[2]), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        total_objects_tracking+=1
        tracking_res.append({"image_id" : frameCount, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : int(box[1])})
    
    out_tracking.write(frame)
    
    frameCount+=1
    prev_frame=frame
# cleanup the camera and close any open windows



out_tracking.release()

print(total_objects_tracking)
with open('ssd_300_keypoint_graal1.json', 'w') as outfile:  
    json.dump(tracking_res, outfile)

1032
778
prev_count   current_count   entered   exited
391


In [9]:
print(frameCount)

801


## Kalman Filter Prediction

In [66]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2 as cv
import time
import json
from bounding_box_utils.bounding_box_utils import iou
feature_params = dict( maxCorners = 25,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
no_tracking_res = [] 
tracking_res = []

# initialize the first frame in the video stream
firstFrame = None
frameCount =0
step = 0.05
total_objects_no_tracking=0
total_objects_tracking=0
frame_width = 640
frame_height = 464
print(frame_width)
print(frame_height)
preds = []
pred = None
tracks=[]

out_tracking = cv.VideoWriter('ssd_300_kalman.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
started = False
# your code
print("prev_count   current_count   entered   exited")
multiplier=0
cc=0
prev_frame=None
kalman_trackers = []
def add_kalman(initialpred):
    
    kalman = cv.KalmanFilter(4,2)
    kalman.measurementMatrix = np.array([[1,0,0,0],
                                     [0,1,0,0]],np.float32)

    kalman.transitionMatrix = np.array([[1,0,1,0],
                                    [0,1,0,1],
                                    [0,0,1,0],
                                    [0,0,0,1]],np.float32)

    kalman.processNoiseCov = np.array([[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,0],
                                   [0,0,0,1]],np.float32) * 0.001
    kalman.predict()
    kalman.correct(initialpred)
    kalman.predict()
    kalman.correct(initialpred)
    kalman.predict()
    kalman.correct(initialpred)
    kalman.predict()
    kalman.correct(initialpred)
    kalman_trackers.append(kalman)
while frameCount<641:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    frame = cv.imread('../modd/%s.jpg'%str(frameCount+1).zfill(5))
    
    if frame is None:
        break
    frameCount+=1
    if(frameCount<0):
        continue
   
    preds = []
    if 'modd/%s.jpg'%str(frameCount+1).zfill(5) in detections:
        
        for box in detections['modd/%s.jpg'%str(frameCount+1).zfill(5)]:
           
            if(box[0]!=4):

                continue
            temp_pred = np.insert(box,0,0)
            temp_pred=np.insert(temp_pred,7,0)
            preds.append(temp_pred)

    for track in tracks:
        track[7]=0
    #print(len(prev_pred),len(temp),len(entered_pred),len(exited_pred))
    
    for pred in preds:
        
        index = box_matches_list(pred,tracks)
        if(index>-1):
            
            multiplier = tracks[index][0]
            org_conf =tracks[index][2]
            if(multiplier<0):
                multiplier=1
            else:
                multiplier +=1
            
             
            
            kalman_trackers[index].correct(center(pred));
            tracks[index][2] = org_conf+ multiplier*step
            tracks[index][3] = pred[3]
            tracks[index][4] = pred[4]
            tracks[index][5] = pred[5]
            tracks[index][6] = pred[6]
            tracks[index][0] = multiplier
            #print(pred)
            
        else:
            print('failed to match')
            print(pred)
            print('in')
            print(tracks)
            temp_pred = np.copy(pred)
           
            multiplier = 1
            print("new object detected")
            print(temp_pred)
            #print("at frame %d"%frameCount)
           
            temp_pred[0] = multiplier
            temp_pred[7]=1
            tracks.append(temp_pred)
            add_kalman(center(temp_pred))
            
            cc = frameCount
    predictions = [i.predict() for i in kalman_trackers]
    
    i=0
    for track in tracks:
        if(track[7]==0):
            #print('not found in current frame')
            track[0]=-1
            #print('multiplier decreased to ',track[0])
            track[2]+=(step*track[0])
            print(predictions[i])
            predicted_center= np.array([predictions[i][0][0],predictions[i][1][0]],np.float32)
            offset = predicted_center- center(track)
            print('for track %d , and kalman %d the predicted offset is'%(i,i))
            print(predictions)
        
            print(offset)
            track[3] += offset[0]
            track[5]+=offset[0]
            track[4]+=offset[1]
            track[6]+=offset[1]
        i+=1
            
            
    to_display = [track for track in tracks if track[2]>0.4]
    incs = [i for i in range(len(tracks)) if tracks[i][2] >0.4]
    
    
    
    if(len(incs)>0 and len(tracks)>0):
        
        kalman_trackers = np.take(kalman_trackers,incs).tolist()
        predictions = np.take(predictions,incs,0).tolist()
    
    #print(len(incs))
    tracks= to_display
    print(len(tracks),len(predictions),len(kalman_trackers))
    #print(tracks)
    i=0
    for box in to_display:
    # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
    
        xmin = int(box[3])
        ymin = int(box[4])
        xmax =int(box[5])
        ymax =int(box[6])
        cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), (0, 255, 0), 2)
        cv.putText(frame,'{}: {:.2f}'.format(classes[int(box[1])], box[2]), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv.circle(frame,(int(predictions[i][0][0]),int(predictions[i][1][0])),5,(255,0,0),2) 
            
            
        total_objects_tracking+=1
        tracking_res.append({"image_id" : frameCount, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : int(box[1])})
        i+=1
    #print('preds%d'%len(preds))
    #print('tracks%d'%len(tracks))
    #print('displayed%d'%len(to_display))
    # if the `q` key is pressed, break from the lop
    #plt.figure()
    #plt.imshow(frame)
    out_tracking.write(frame)
    #cv.imwrite("512_frames/frame%d.jpg"%frameCount,frame)
    prev_frame=frame
# cleanup the camera and close any open windows



out_tracking.release()

print(total_objects_tracking)
with open('ssd_300_kalman.json', 'w') as outfile:  
    json.dump(tracking_res, outfile)

640
464
prev_count   current_count   entered   exited
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
failed to match
[  0.         4.         0.689186  -6.       202.       112.
 351.         0.      ]
in
[]
new object detected
[  0.         4.         0.689186  -6.       202.       112.
 351.         0.      ]
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 

1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
[[ 2.4690288e+02]
 [ 2.0367050e+02]
 [ 1.0145302e+00]
 [-5.5540405e-02]]
for track 0 , and kalman 0 the predicted offset is
[array([[ 2.4690288e+02],
       [ 2.0367050e+02],
       [ 1.0145302e+00],
       [-5.5540405e-02]], dtype=float32)]
[2.4028778 4.1705017]
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
[[259.55865   ]
 [200.32973   ]
 [  1.0497717 ]
 [  0.44862708]]
for track 0 , and kalman 0 the predicted offset is
[array([[259.55865   ],
       [200.32973   ],
       [  1.0497717 ],
       [  0.44862708]], dtype=float32)]
[ 1.5586548 -5.670273 ]
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
[[267.95154   ]
 [208.50215   ]
 [  0.83503187]
 [  0.5692753 ]]
for track 0 , and kalman 0 the predicted offset is
[array([[267.95154   ],
       [208.50215   ],
       [  0.83503187],
       [  0.5692753 ]], dtype=float32)]
[1.9515381 2.0021515]
1 1 1
[[268.78656   ]
 [209.07143   ]
 [  0.83503187]
 [  

[[308.28433   ]
 [190.9134    ]
 [  0.97261286]
 [ -0.38512695]]
for track 0 , and kalman 0 the predicted offset is
[array([[308.28433   ],
       [190.9134    ],
       [  0.97261286],
       [ -0.38512695]], dtype=float32)]
[ 0.97262573 -0.38513184]
1 1 1
[[309.25696   ]
 [190.52827   ]
 [  0.97261286]
 [ -0.38512695]]
for track 0 , and kalman 0 the predicted offset is
[array([[309.25696   ],
       [190.52827   ],
       [  0.97261286],
       [ -0.38512695]], dtype=float32)]
[ 0.97262573 -0.38513184]
1 1 1
[[310.22958   ]
 [190.14314   ]
 [  0.97261286]
 [ -0.38512695]]
for track 0 , and kalman 0 the predicted offset is
[array([[310.22958   ],
       [190.14314   ],
       [  0.97261286],
       [ -0.38512695]], dtype=float32)]
[ 0.97262573 -0.38513184]
1 1 1
[[311.2022    ]
 [189.75801   ]
 [  0.97261286]
 [ -0.38512695]]
for track 0 , and kalman 0 the predicted offset is
[array([[311.2022    ],
       [189.75801   ],
       [  0.97261286],
       [ -0.38512695]], dtype=float32)]


[[345.2441    ]
 [176.2784    ]
 [  0.97261286]
 [ -0.38512695]]
for track 0 , and kalman 0 the predicted offset is
[array([[345.2441    ],
       [176.2784    ],
       [  0.97261286],
       [ -0.38512695]], dtype=float32)]
[ 0.97262573 -0.38513184]
1 1 1
[[346.21674   ]
 [175.89326   ]
 [  0.97261286]
 [ -0.38512695]]
for track 0 , and kalman 0 the predicted offset is
[array([[346.21674   ],
       [175.89326   ],
       [  0.97261286],
       [ -0.38512695]], dtype=float32)]
[ 0.97262573 -0.38513184]
1 1 1
[[347.18936   ]
 [175.50813   ]
 [  0.97261286]
 [ -0.38512695]]
for track 0 , and kalman 0 the predicted offset is
[array([[347.18936   ],
       [175.50813   ],
       [  0.97261286],
       [ -0.38512695]], dtype=float32)]
[ 0.97262573 -0.38513184]
1 1 1
[[348.162     ]
 [175.123     ]
 [  0.97261286]
 [ -0.38512695]]
for track 0 , and kalman 0 the predicted offset is
[array([[348.162     ],
       [175.123     ],
       [  0.97261286],
       [ -0.38512695]], dtype=float32)]


1 1 1
[[4.2579874e+02]
 [1.9631863e+02]
 [7.7103919e-01]
 [9.4845124e-02]]
for track 0 , and kalman 0 the predicted offset is
[array([[4.2579874e+02],
       [1.9631863e+02],
       [7.7103919e-01],
       [9.4845124e-02]], dtype=float32)]
[ 0.7987366 -1.181366 ]
1 1 1
[[4.2656976e+02]
 [1.9641348e+02]
 [7.7103919e-01]
 [9.4845124e-02]]
for track 0 , and kalman 0 the predicted offset is
[array([[4.2656976e+02],
       [1.9641348e+02],
       [7.7103919e-01],
       [9.4845124e-02]], dtype=float32)]
[0.7710266  0.09484863]
1 1 1
[[4.2734079e+02]
 [1.9650833e+02]
 [7.7103919e-01]
 [9.4845124e-02]]
for track 0 , and kalman 0 the predicted offset is
[array([[4.2734079e+02],
       [1.9650833e+02],
       [7.7103919e-01],
       [9.4845124e-02]], dtype=float32)]
[0.7710266  0.09484863]
1 1 1
[[4.2811182e+02]
 [1.9660318e+02]
 [7.7103919e-01]
 [9.4845124e-02]]
for track 0 , and kalman 0 the predicted offset is
[array([[4.2811182e+02],
       [1.9660318e+02],
       [7.7103919e-01],
       [9

[[4.4981287e+02]
 [2.0393532e+02]
 [4.1940632e-01]
 [5.3230423e-01]]
for track 0 , and kalman 0 the predicted offset is
[array([[4.4981287e+02],
       [2.0393532e+02],
       [4.1940632e-01],
       [5.3230423e-01]], dtype=float32)]
[-0.18713379 -3.564682  ]
1 1 1
[[4.5023227e+02]
 [2.0446762e+02]
 [4.1940632e-01]
 [5.3230423e-01]]
for track 0 , and kalman 0 the predicted offset is
[array([[4.5023227e+02],
       [2.0446762e+02],
       [4.1940632e-01],
       [5.3230423e-01]], dtype=float32)]
[0.41940308 0.53230286]
1 1 1
[[4.5065167e+02]
 [2.0499992e+02]
 [4.1940632e-01]
 [5.3230423e-01]]
for track 0 , and kalman 0 the predicted offset is
[array([[4.5065167e+02],
       [2.0499992e+02],
       [4.1940632e-01],
       [5.3230423e-01]], dtype=float32)]
[0.41940308 0.53230286]
1 1 1
[[4.5107108e+02]
 [2.0553223e+02]
 [4.1940632e-01]
 [5.3230423e-01]]
for track 0 , and kalman 0 the predicted offset is
[array([[4.5107108e+02],
       [2.0553223e+02],
       [4.1940632e-01],
       [5.323

1 1 1
1 1 1
1 1 1
1 1 1
[[ 5.0301221e+02]
 [ 2.4164040e+02]
 [-1.3576771e+00]
 [ 1.6187546e-01]]
for track 0 , and kalman 0 the predicted offset is
[array([[ 5.0301221e+02],
       [ 2.4164040e+02],
       [-1.3576771e+00],
       [ 1.6187546e-01]], dtype=float32)]
[-3.487793   1.6403961]
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
[[ 5.4763013e+02]
 [ 2.4727194e+02]
 [ 2.6393087e+00]
 [-2.4646120e-02]]
for track 0 , and kalman 0 the predicted offset is
[array([[ 5.4763013e+02],
       [ 2.4727194e+02],
       [ 2.6393087e+00],
       [-2.4646120e-02]], dtype=float32)]
[ 1.630127   -0.22805786]
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
1 1 1
[[603.4082   ]
 [271.57965  ]
 [  2.8215559]
 [  1.1744863]]
for track 0 , and kalman 0 the predicted offset is
[array([[603.4082   ],
       [271.57965  ],
       [  2.8215559],
       [  1.1744863]], dtype=float32)]
[  4.408203 -12.420349]
1 

In [18]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2 as cv
import time
import json
import numpy as np
from bounding_box_utils.bounding_box_utils import iou
feature_params = dict( maxCorners = 25,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
no_tracking_res = [] 
tracking_res = []

# initialize the first frame in the video stream
firstFrame = None
frameCount =70
step = 0.05
total_objects_no_tracking=0
total_objects_tracking=0
frame_width = 640
frame_height = 464
print(frame_width)
print(frame_height)
preds = []
pred = None
tracks=[]
kalman_trackers = []
def add_kalman():
    
    kalman = cv.KalmanFilter(4,2)
    kalman.measurementMatrix = np.array([[1,0,0,0],
                                     [0,1,0,0]],np.float32)

    kalman.transitionMatrix = np.array([[1,0,1,0],
                                    [0,1,0,1],
                                    [0,0,1,0],
                                    [0,0,0,1]],np.float32)

    kalman.processNoiseCov = np.array([[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,0],
                                   [0,0,0,1]],np.float32) * 0.03
    kalman_trackers.append(kalman)

out_tracking = cv.VideoWriter('ssd_300_kalman.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
started = False
# your code

multiplier=0
cc=0
prev_frame=None

while frameCount<641:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    frame = cv.imread('../modd/%s.jpg'%str(frameCount+1).zfill(5))
    
 
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    
    if(frameCount<0):
        continue
    elif started==False:
        start_time = time.time()
        started=True
    # resize the frame, convert it to grayscale, and blur it
    
    
    
        
    # loop over the contours
    preds = []
    if 'modd/%s.jpg'%str(frameCount+1).zfill(5) in detections:
        
        for box in detections['modd/%s.jpg'%str(frameCount+1).zfill(5)]:
           
            if(box[0]!=4):

                continue
            temp_pred = np.insert(box,0,0)
            temp_pred=np.insert(temp_pred,7,0)
            preds.append(temp_pred)
   
    for track in tracks:
        track[7]=0
    
    for pred in preds:
        
        index = box_matches_list(pred,tracks)
        if(index>-1):
            
            multiplier = tracks[index][0]
            
            org_conf =tracks[index][2]
            if(multiplier<0):
                multiplier=1
            else:
                
                multiplier +=1
                
             
            del tracks[index]
            kalman_trackers[index].correct(center(pred));

          
            
    
            pred[2] = org_conf+ multiplier*step
            
            pred[0] = multiplier
            #print(pred)
            
            tracks.append(pred)
        else:
            
            temp_pred = np.copy(pred)
           
            multiplier = 1
            
            add_kalman()
            print('added')
            temp_pred[0] = multiplier
            temp_pred[2] = 1
            
            tracks.append(temp_pred)
            
            cc = frameCount
    
    predictions = [i.predict() for i in kalman_trackers]
    
    i=0
    for track in tracks:
        if(track[7]!=1):
            if(box_matches_list(track,preds)==-1):
                #print('not found in current frame')
                track[0]=-1
            #print('multiplier decreased to ',track[0])
                track[2]+=(step*track[0])
                predicted_center= np.array([predictions[i][0][0],predictions[i][1][0]],np.float32)
                offset = predicted_center- center(track)
                
                track[3] += offset[0]
                track[5]+=offset[0]
                track[4]+=offset[1]
                track[6]+=offset[1]
        i+=1
            
    
    to_display = [track for track in tracks if track[2]>0.4]
    tracks=to_display
    #print(len(to_display))
    i=0
    for box in to_display:
        
    # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
        
        xmin = int(box[3] )
        ymin = int(box[4])
        xmax =int(box[5])
        ymax =int(box[6])
        cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), (0, 255, 0), 2)
        cv.putText(frame,'{}: {:.2f}'.format(classes[int(box[1])], box[2]), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv.circle(frame,(predictions[i][0][0],predictions[i][1][0]),5,(255,0,0),2)
        total_objects_tracking+=1
        tracking_res.append({"image_id" : frameCount, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : int(box[1])})
        i+=1
    cv.imwrite("one.jpg",frame)
    out_tracking.write(frame)
    prev_frame=frame
    frameCount+=1
# cleanup the camera and close any open windows
elapsed_time = time.time() - start_time
print(elapsed_time)

cv.destroyAllWindows()

out_tracking.release()

print(total_objects_tracking)
with open('ssd_300_kalman.json', 'w') as outfile:  
    json.dump(tracking_res, outfile)

640
464
added
1
1
1
1
1
1
1
1
1
added
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
added
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
added
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
added
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
added
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
added
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
added
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
added
4
4
4
4
4
4
4
added
5
5
5
5
5
5
5
5
5
5
5
5
added
6
6
6
6
6
6
added
7
7
7
7
7
7
7
7
6
6
6
6
6
6
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [11]:
print(frameCount)

70


## General Tracking

In [67]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2 as cv
import time
import json
from bounding_box_utils.bounding_box_utils import iou
feature_params = dict( maxCorners = 25,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
no_tracking_res = [] 
tracking_res = []

# initialize the first frame in the video stream
firstFrame = None
frameCount =0
step = 0.05
total_objects_no_tracking=0
total_objects_tracking=0
frame_width = 640
frame_height = 464
print(frame_width)
print(frame_height)
preds = []
pred = None
tracks=[]

out_tracking = cv.VideoWriter('ssd_300_modd_kalman_twopoints.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
started = False
# your code
print("prev_count   current_count   entered   exited")
multiplier=0
cc=0
prev_frame=None
kalman_trackers = []
def add_kalman(initialpred1,initialpred2):
    
    kalman1 = cv.KalmanFilter(4,2)
    kalman1.measurementMatrix = np.array([[1,0,0,0],
                                     [0,1,0,0]],np.float32)

    kalman1.transitionMatrix = np.array([[1,0,1,0],
                                    [0,1,0,1],
                                    [0,0,1,0],
                                    [0,0,0,1]],np.float32)

    kalman1.processNoiseCov = np.array([[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,0],
                                   [0,0,0,1]],np.float32) * 0.001
    kalman1.predict()
    kalman1.correct(initialpred1)
    kalman1.predict()
    kalman1.correct(initialpred1)
    kalman1.predict()
    kalman1.correct(initialpred1)
    kalman1.predict()
    kalman1.correct(initialpred1)
    kalman2 = cv.KalmanFilter(4,2)
    kalman2.measurementMatrix = np.array([[1,0,0,0],
                                     [0,1,0,0]],np.float32)

    kalman2.transitionMatrix = np.array([[1,0,1,0],
                                    [0,1,0,1],
                                    [0,0,1,0],
                                    [0,0,0,1]],np.float32)

    kalman2.processNoiseCov = np.array([[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,0],
                                   [0,0,0,1]],np.float32) * 0.001
    kalman2.predict()
    kalman2.correct(initialpred2)
    kalman2.predict()
    kalman2.correct(initialpred2)
    kalman2.predict()
    kalman2.correct(initialpred2)
    kalman2.predict()
    kalman2.correct(initialpred2)
    
    kalman_trackers.append([kalman1,kalman2])
    
while frameCount<641:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    frame = cv.imread('../modd/%s.jpg'%str(frameCount+1).zfill(5))
    
    if frame is None:
        break
    frameCount+=1
    if(frameCount<0):
        continue
   
    preds = []
    if 'modd/%s.jpg'%str(frameCount+1).zfill(5) in detections:
        
        for box in detections['modd/%s.jpg'%str(frameCount+1).zfill(5)]:
           
            if(box[0]!=4):

                continue
            temp_pred = np.insert(box,0,0)
            temp_pred=np.insert(temp_pred,7,0)
            preds.append(temp_pred)

    for track in tracks:
        track[7]=0
    #print(len(prev_pred),len(temp),len(entered_pred),len(exited_pred))
    
    for pred in preds:
        
        index = box_matches_list(pred,tracks)
        if(index>-1):
            
            multiplier = tracks[index][0]
            org_conf =tracks[index][2]
            if(multiplier<0):
                multiplier=1
            else:
                multiplier +=1
            
             
            
            kalman_trackers[index][0].correct(topleft(pred));
            kalman_trackers[index][1].correct(botright(pred));
            tracks[index][2] = org_conf+ multiplier*step
            tracks[index][3] = pred[3]
            tracks[index][4] = pred[4]
            tracks[index][5] = pred[5]
            tracks[index][6] = pred[6]
            tracks[index][0] = multiplier
            #print(pred)
            
        else:
            
            temp_pred = np.copy(pred)
           
            multiplier = 1
            
            #print("at frame %d"%frameCount)
           
            temp_pred[0] = multiplier
            temp_pred[7]=1
            tracks.append(temp_pred)
            add_kalman(topleft(temp_pred),botright(temp_pred))
            
            cc = frameCount
    predictions = [[i[0].predict(),i[1].predict()] for i in kalman_trackers]
    
    i=0
    for track in tracks:
        if(track[7]==0):
            #print('not found in current frame')
            track[0]=-1
            #print('multiplier decreased to ',track[0])
            track[2]+=(step*track[0])
            #print(predictions[i])
            
            track[3] = predictions[i][0][0][0]
            track[4]=predictions[i][0][1][0]
            track[5]=predictions[i][1][0][0]
            track[6]=predictions[i][1][1][0]
        i+=1
            
            
    to_display = [track for track in tracks if track[2]>0.4]
    incs = [i for i in range(len(tracks)) if tracks[i][2] >0.4]
    
    
    if(len(incs)>0 and len(tracks)>0):
        
        kalman_trackers = np.take(kalman_trackers,incs,0).tolist()
        predictions = np.take(predictions,incs,0).tolist()
    
    
    #print(len(incs))
    tracks= to_display
    
    #print(tracks)
    i=0
    for box in to_display:
    # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
    
        xmin = int(box[3])
        ymin = int(box[4])
        xmax =int(box[5])
        ymax =int(box[6])
        cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), (0, 255, 0), 2)
        cv.putText(frame,'{}: {:.2f}'.format(classes[int(box[1])], box[2]), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv.circle(frame,(int(predictions[i][0][0][0]),int(predictions[i][0][1][0])),5,(255,0,0),2)
        cv.circle(frame,(int(predictions[i][1][0][0]),int(predictions[i][1][1][0])),5,(255,0,0),2)    
            
        total_objects_tracking+=1
        tracking_res.append({"image_id" : frameCount, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : int(box[1])})
        i+=1
    #print('preds%d'%len(preds))
    #print('tracks%d'%len(tracks))
    #print('displayed%d'%len(to_display))
    # if the `q` key is pressed, break from the lop
    #plt.figure()
    #plt.imshow(frame)
    out_tracking.write(frame)
    cv.imwrite("512_frames/frame%d.jpg"%frameCount,frame)
    prev_frame=frame
# cleanup the camera and close any open windows



out_tracking.release()

print(total_objects_tracking)
with open('ssd_300_modd_kalman_twopoints.json', 'w') as outfile:  
    json.dump(tracking_res, outfile)

640
464
prev_count   current_count   entered   exited
581
